In [1]:
import surprise 

print(surprise.__version__)

1.1.1


In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
data=Dataset.load_builtin('ml-100k')
trainset, testset=train_test_split(data,test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\Lee/.surprise_data/ml-100k


In [4]:
algo = SVD()
algo.fit(trainset)

In [5]:
predictions = algo.test(testset)
predictions[:5]

[Prediction(uid='120', iid='282', r_ui=4.0, est=3.847880569808352, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.933807762061793, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.877511406381616, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.6450770990029637, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.5412336016934507, details={'was_impossible': False})]

In [6]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.847880569808352),
 ('882', '291', 3.933807762061793),
 ('535', '507', 3.877511406381616)]

In [7]:
pred = algo.predict(str(196),str(302))
print(pred)

user: 196        item: 302        r_ui = None   est = 4.07   {'was_impossible': False}


In [8]:
accuracy.rmse(predictions)

RMSE: 0.9472


0.9472026641321752

In [10]:
import pandas as pd

ratings = pd.read_csv('./ratings.csv')
ratings.to_csv('./ratings_noh.csv', index=False,header=False)

In [11]:
from surprise import Reader

reader = Reader(line_format = 'user item rating timestamp', sep=',',rating_scale=(0.5,5))
data=Dataset.load_from_file('./ratings_noh.csv', reader=reader)

In [12]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [13]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./ratings.csv')
reader=Reader(rating_scale=(0.5,5.0))
data=Dataset.load_from_df(ratings[['userId','movieId','rating']],reader=reader)

trainset, testset = train_test_split(data, test_size=.25, random_state=0)
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [15]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('./ratings.csv')
reader=Reader(rating_scale=(0.5,5.0))
data=Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)

algo = SVD(random_state=0)
cross_validate(algo,data,measures=['RMSE','MAE'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8691  0.8721  0.8738  0.8774  0.8764  0.8738  0.0030  
MAE (testset)     0.6684  0.6699  0.6717  0.6748  0.6720  0.6713  0.0022  
Fit time          6.47    6.33    6.59    6.32    6.42    6.43    0.10    
Test time         0.16    0.29    0.27    0.16    0.15    0.21    0.06    


{'test_rmse': array([0.86909827, 0.87212853, 0.8737937 , 0.87736955, 0.87636227]),
 'test_mae': array([0.66835857, 0.66985014, 0.67165596, 0.67477623, 0.67197477]),
 'fit_time': (6.4663286209106445,
  6.3298375606536865,
  6.5853612422943115,
  6.323483467102051,
  6.4239418506622314),
 'test_time': (0.16255426406860352,
  0.2863762378692627,
  0.2659027576446533,
  0.16149616241455078,
  0.15340232849121094)}

In [17]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs':[20,40], 'n_factors':[50,100]}

gs = GridSearchCV(SVD,param_grid,measures=['rmse','mae'],cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8762998285645093
{'n_epochs': 20, 'n_factors': 50}


In [20]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',',rating_scale=(0.5,5.0))
data_folds = DatasetAutoFolds(ratings_file='./ratings_noh.csv',reader=reader)

trainset=data_folds.build_full_trainset()

In [21]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [27]:
movies = pd.read_csv('./movies.csv')
pred = algo.predict(str(9),str(42),verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [29]:
def get_unseen_surprise(ratings,movies,userId):
    seen_movies=ratings[ratings['userId']==userId]['movieId'].tolist()
    total_movies = movies['movieId'].tolist()
    
    unseen_movies=[movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
        
    return unseen_movies



평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [34]:
def recomm_movie_by_surprise(algo, userId,unseen_movies,top_n=10):
    predictions = [algo.predict(str(userId),str(movieId)) for movieId in unseen_movies]
    
    def sortkey_est(pred):
        return pred.est
    
    predictions.sort(key=sortkey_est,reverse=True)
    top_predictions=predictions[:top_n]
    
    top_movie_ids=[int(pred.iid) for pred in top_predictions]
    top_movie_rating =[pred.est for pred in top_predictions]
    top_movie_titles=movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds=[(id,title,rating) for id,title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies,9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

for top_movie in top_movie_preds:
    print(top_movie[1],":",top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
